# leafGAN_GA

Use a genetic algorithm with a surrogate model evaluator to generate optimized structures from the StyleGAN latent space

Sabrina Shen, MIT, 2022

Shen, S.C., Buehler, M.J. Nature-inspired Architected materials using unsupervised deep learning. Accepted (2022).|

In [1]:
#Import packages
import os, sys

import subprocess
from subprocess import PIPE

from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance_matrix
import matplotlib.pyplot as mpplt
import matplotlib
from math import *

In [2]:
# pytorch packages
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten
from torch import nn 
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
import torchvision.transforms as transforms
import torch.nn as nn

In [3]:
import random
from pathlib import Path
from time import strftime
import PIL 
from PIL import Image
from PIL import Image, ImageOps, ImageFilter
import skimage.io
import skimage.filters
import pandas as pd
import pickle
import numpy as np
import ipywidgets as widgets
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
import torch
import glob
import shutil
import generate
import dnnlib
import torch_utils
import legacy

In [5]:
!nvidia-smi -L
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

GPU 0: Quadro RTX 5000 (UUID: GPU-78162963-0a30-bd79-76cb-1dd6b2a4206c)


In [6]:
#Lattice param

#spacing of lattice
r0_lattice=1.12246
#r0_lattice=.93*2


#hexagonal
r0_lattice_x=r0_lattice*2
r0_lattice_y=r0_lattice * sqrt(3. )
#r0_lattice_x=2.
#r0_lattice_y=2

#2 atoms per unit cell
ucellsize=4
ux = [0, r0_lattice/2,r0_lattice, r0_lattice*3/2]
uy = [0, r0_lattice*sqrt(3)/2.,0, r0_lattice*sqrt(3)/2.   ]


In [7]:
#StyleGAN support functions

def load_model(model_path):
    with dnnlib.util.open_url(model_path) as f:
        G = legacy.load_network_pkl(f)['G_ema'].to(device)
    return G

# Generate images given a random seed (Integer)
def generate_rand_image(seed):
    label = torch.zeros([1, G.c_dim], device=device)
    
    z = torch.from_numpy(np.random.RandomState(seed).randn(1, G.z_dim)).to(device)
    img = G(z, label, truncation_psi=1, noise_mode='const')
    img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
    #PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{outdir}/seed{seed:04d}.png')
    return img, z

# Generate images given a latent code ( vector of size [1, 512] )
def generate_image_from_z(z):
    label = torch.zeros([1, G.c_dim], device=device)
    z = torch.tensor(z)
    z = z.to(device)

    img = G(z, label, truncation_psi=1, noise_mode='const')
    img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
    img = PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB')#.save(f'{outdir}/seed{seed:04d}.png')
    return img

# Interpolate an image between two others
def linear_interpolate(code1, code2, alpha):
    return code1 * alpha + code2 * (1 - alpha)

def generate_image_from_w(w):
    img = G.synthesis(w, noise_mode='const')
    img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
    img = PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB')#.save(f'{outdir}/proj{idx:02d}.png')
    return img

def z_to_w(z):
    label = torch.zeros([1, G.c_dim], device=device)
    z = torch.from_numpy(z).to(device)
    w = G.mapping(z, label, truncation_psi=1, truncation_cutoff=8)
    return w


In [8]:
#Load Trained StyleGAN Model
model_path = './leaf-network-snapshot.pkl'
#Gs, noise_vars, Gs_kwargs = load_model(model_path)

G=load_model(model_path)

In [70]:
#Evaluator model function with CNN

#Load trained cnn surrogate
model = GA_functions.leaf_cnn()
model.load_state_dict(torch.load('GA/trained_models/4layer.pth'))
model.to(device)
model.eval()

save_folder='./GA/img_ratio_cnn2'

def evaluate(gen,i,z):
    folder_pth = Path(f'{save_folder}/{gen}')
    if not folder_pth.exists():
        folder_pth.mkdir()
    img_pth = f'{save_folder}/{gen}/{i}.png'
    img_temp = generate_image_from_w(z)
    img_temp.save(img_pth)
    unit, density = GA_functions.process_to_unit(img_pth)
    cv2.imwrite(img_pth,unit)
    
    #check for connectivity
    if GA_functions.check_border(img_pth):      
        img = Image.open(img_pth)
        transform = transforms.ToTensor()
        img = transform(img)
        img = img.unsqueeze(0)
        #img.to(device)
        pred = model(img.to(device))/100
    else:
        return (0,1)
    
    return (pred.cpu().detach().numpy()[0][0], density)

In [71]:
Target = 10.0

def fit_fx(mod,dens):
    #equal weighted GA
    #fit = 0.5*mod-0.5*dens
    
    #random weighted GA
    #weight = random.random()
    #fit = weight*mod-(1-weight)*dens
    
    #modulus:density ratio
    fit = mod/dens
    
    #maximize modulus
    #fit = mod
    
    fit = -1*abs(fit - Target)
    return fit

In [72]:
################ Initialization using w latent space######################

#Generate Initial Population
Population_Size = 10
top_pop_size = 50

fitness=[]
top_pop=[] #A list of arrays
top_mod=[]

Population=[] #A list of arrays
for i in range(top_pop_size):
    z = np.random.RandomState(i).randn(1, G.z_dim)#.to(device)
    w = z_to_w(z)
    mod, dens = evaluate('top_pop',i,w)
    if i < Population_Size:
        fit = fit_fx(mod, dens)
        fitness.append(fit)
        Population.append(w)
    top_pop.append([mod, dens, fit])
    top_mod.append([mod, dens, fit])
    shutil.copyfile(f'{save_folder}/top_pop/{i}.png', f'{save_folder}/top_mod/{i}.png')

In [73]:
################ GENETIC ALGORITHM START WITH W LATENT SPACE ######################

#Choose how many generations to run
generations=1000 #1000 gen ~ 3.5 hours
bestmod=[]
bestdens=[]
bestfit=[]
#avgmod=[]
#devmod=[]

#best_idx=0
convergence=10
avg_fitness=[]
all_fitness=[]
all_mod=[]
all_dens=[]

for gen in range(generations):
    ###Selection (input Population_Size, Population; output Parent1, Parent2)

    sel_prob=[x-min(fitness) for x in fitness]
    sel_prob=np.array(sel_prob)/sum(sel_prob)
    sel=np.random.choice(range(0, Population_Size),size=2,replace=False,p=sel_prob)
    
    Parent1=Population[sel[0]]
    Parent2=Population[sel[1]]

    Child1=linear_interpolate(Parent1, Parent2, random.gauss(0.5, 0.2))
    Child2=linear_interpolate(Parent1, Parent2, random.gauss(1.5, 0.2))
    Child3=Parent1
    for i in range(len(Child1[0])):
        if random.random()<0.5:
            Child3[0][i]=Parent2[0][i]
            
    ###Mutation (input Child1; output MChild)
    MChild=Child1
    PointRate = 1/convergence #percentage chance of mutation
    PointMag = random.gauss(2, 1)/convergence #stddev of mutation
    for i in range(len(Child1[0])):
        if random.random()<PointRate:
            MChild[0][i]=torch.from_numpy(np.random.default_rng().uniform(-5,5,(1,512)))
    
    MChild2=Parent1*random.gauss(1.5, 0.2)
    
    RChild = z_to_w(np.random.default_rng().uniform(-5,5,(1,512)))
    RChild2 = z_to_w(np.random.default_rng().uniform(-5,5,(1,512)))

    for j in (Child1,Child2,Child3,MChild,MChild2,RChild, RChild2):
        Population.append(j)

    ###Competition 
    modulus = []
    density = []
    fitness=[]
    
    for i,z in enumerate(Population):
        mod, dens = evaluate(gen,i,z)
        modulus.append(mod)
        density.append(dens)
        fitness.append(fit_fx(mod,dens))
        top_min=np.argmin(np.array(top_pop)[:,2])
        
        #mod - dens
        #if fit > np.min(np.array(top_pop)[:,2]) and i>9: 
            #top_pop[top_min]=[mod,dens,fit]
            #shutil.copyfile(f'./GA/img2/{gen}/{i}.png', f'./GA/img2/top/{top_min}.png')
        
        #pareto front
        if i>9:
            for j in range(len(top_pop)):
                if mod > top_pop[j][0] and dens < top_pop[j][1]:
                    top_pop[j] = [mod,dens,fit]
                    shutil.copyfile(f'{save_folder}/{gen}/{i}.png', f'{save_folder}/top_pop/{j}.png')
                    break

            if mod > np.min(np.array(top_mod)[:,0]):
                k = np.argmin(np.array(top_mod)[:,0])
                top_mod[k] = [mod,dens,fit]
                shutil.copyfile(f'{save_folder}/{gen}/{i}.png', f'{save_folder}/top_mod/{k}.png')
                
    print(fitness)

    Safe_Index=np.argmax(fitness)    

    Random_Death=3

    rand_idx=random.sample([i for i in range(len(fitness)) if i not in [Safe_Index]],k=Random_Death)
    
    Attrition=4
    min_idx=[]

    for k in range(Attrition):
        idxs=[i for i in range(len(fitness)) if i not in [Safe_Index] and i not in min_idx and i not in rand_idx]
        min_idx.append(idxs[np.argmin([fitness[i] for i in idxs])])

        
    attr_idx=np.array(rand_idx+min_idx)
    attr_idx[::-1].sort()
    
    for i in attr_idx:
        print("Structure "+str(i)+" dies with "+str(fitness[i])+" fitness")
        Population.pop(i)
        fitness.pop(i) 
        modulus.pop(i)
        density.pop(i)
        os.remove(f"{save_folder}/{gen}/{i}.png")

    Safe_Index=np.argmax(fitness)
        
    bestmod.append(modulus[Safe_Index])
    bestdens.append(density[Safe_Index])
    bestfit.append(fitness[Safe_Index])
    
    avg_fitness.append(sum(fitness) / len(fitness))
    all_fitness.append(fitness)
    all_mod.append(modulus)
    all_dens.append(density)
    
    print("Generation "+str(gen)+" completed. Best modulus: "+str(max(modulus))+"\n")

    #convergence = abs(max(fitness)-min(fitness))

    #if convergence < 0.001 and gen > generations/4:
    #    print("Structure Saturation")
    #    break

[-9.801224755224393, -9.711375077037992, -9.792282661238447, -9.493185301494535, -9.791474152857283, -9.779403670243674, -10.0, -9.802780163271375, -9.760263770672639, -9.76376520949514, -10.0, -10.0, -9.802946974286648, -10.0, -9.806530228800368, -9.810570994114746, -9.8008095518157]
Structure 15 dies with -9.810570994114746 fitness
Structure 13 dies with -10.0 fitness
Structure 12 dies with -9.802946974286648 fitness
Structure 11 dies with -10.0 fitness
Structure 10 dies with -10.0 fitness
Structure 6 dies with -10.0 fitness
Structure 2 dies with -9.792282661238447 fitness
Generation 0 completed. Best modulus: 0.42029366

[-9.800508486104246, -9.826312440814394, -9.49343228294522, -9.791532526082417, -9.779210582954676, -9.802879015654256, -9.760233162275327, -9.763447046416694, -9.806401243071782, -9.801010618875504, -9.674242425636601, -10.0, -9.826375101380428, -9.67422948261877, -9.813205807322019, -9.38561374682961, -9.789126881018259]
Structure 14 dies with -9.813205807322019 f

[-9.307893656333402, -9.307766214693252, -9.28704571209086, -9.293068346528107, -9.277860961586796, -9.295226992755161, -9.279208042951968, -9.306939709647601, -9.30799309690002, -9.280211340152276, -9.298185787440344, -9.274451876295727, -9.279344142827773, -9.298185148997288, -9.30639169856775, -9.469571335496378, -9.792328564817902]
Structure 16 dies with -9.792328564817902 fitness
Structure 15 dies with -9.469571335496378 fitness
Structure 14 dies with -9.30639169856775 fitness
Structure 8 dies with -9.30799309690002 fitness
Structure 7 dies with -9.306939709647601 fitness
Structure 1 dies with -9.307766214693252 fitness
Structure 0 dies with -9.307893656333402 fitness
Generation 12 completed. Best modulus: 0.6621778

[-9.287049226813968, -9.29303285202075, -9.277975112055966, -9.295167176146782, -9.279266455759936, -9.282733784568954, -9.298192108026608, -9.274416348308032, -9.278542687634264, -9.298175955417266, -9.765784136858679, -9.280917575433197, -9.28275925670479, -9.765244

[-9.286503502554197, -9.286315659084023, -9.296892519936042, -9.283194010933776, -9.298512805118929, -9.237324873649172, -9.298658022496673, -9.300843350363591, -9.298514039629076, -9.295593992176771, -9.422082995304539, -9.286356028664242, -9.286460897524911, -9.421970020163373, -9.29925825272156, -9.66766873857412, -9.528665131060308]
Structure 16 dies with -9.528665131060308 fitness
Structure 15 dies with -9.66766873857412 fitness
Structure 13 dies with -9.421970020163373 fitness
Structure 12 dies with -9.286460897524911 fitness
Structure 11 dies with -9.286356028664242 fitness
Structure 10 dies with -9.422082995304539 fitness
Structure 9 dies with -9.295593992176771 fitness
Generation 24 completed. Best modulus: 0.69136053

[-9.286464710938025, -9.286302180641115, -9.296829207820595, -9.283108555660661, -9.298662765614605, -9.237366630929735, -9.298662960537259, -9.300823629743396, -9.298464659223221, -9.29924529353661, -9.557114489631722, -9.301925979742483, -9.283083834215109, -9

[-9.287297160909414, -9.287306632290186, -9.294551909133785, -9.284418343772254, -9.2962467840943, -9.295535034440212, -9.292459187885544, -9.284462345034635, -9.299611220680054, -9.29065070992569, -9.29018347418598, -9.747041170225923, -9.299663502505737, -9.290005235231137, -9.326470719952004, -10.0, -9.769474000717965]
Structure 16 dies with -9.769474000717965 fitness
Structure 15 dies with -10.0 fitness
Structure 14 dies with -9.326470719952004 fitness
Structure 12 dies with -9.299663502505737 fitness
Structure 11 dies with -9.747041170225923 fitness
Structure 8 dies with -9.299611220680054 fitness
Structure 0 dies with -9.287297160909414 fitness
Generation 36 completed. Best modulus: 0.68806744

[-9.287285163827104, -9.294575203743525, -9.29619532385913, -9.296242443528593, -9.295586097288504, -9.292510982235651, -9.294357532755601, -9.290721393372419, -9.289990412736962, -9.290184089227232, -9.29425941764037, -9.290031882356498, -9.296771606098407, -9.294269648878608, -9.30514420

[-9.280205070556704, -9.280112485065228, -9.273339498398613, -9.273311870287582, -9.269911185378211, -9.286309088491457, -9.272879566009049, -9.252487499793128, -9.272976795021046, -9.319088159345583, -9.419675137134309, -9.27289225368305, -9.273105872684111, -9.419687190994953, -9.27259366634061, -9.518492584228516, -9.780270092630584]
Structure 16 dies with -9.780270092630584 fitness
Structure 15 dies with -9.518492584228516 fitness
Structure 13 dies with -9.419687190994953 fitness
Structure 12 dies with -9.273105872684111 fitness
Structure 10 dies with -9.419675137134309 fitness
Structure 9 dies with -9.319088159345583 fitness
Structure 5 dies with -9.286309088491457 fitness
Generation 48 completed. Best modulus: 0.7006791

[-9.280169373425794, -9.280125161961541, -9.273316561853607, -9.273336044884733, -9.271511838452774, -9.272847717357982, -9.252521116402423, -9.272883967855131, -9.272882349529365, -9.272683451677544, -9.95199950128539, -9.404643642934923, -9.271542584893005, -9.

...

[-6.916337339339718, -10.0, -9.278028364210412, -8.408174694943506, -8.408447563842376, -7.078398506596404, -9.241429889131398, -7.629891690340909, -9.277880252458104, -9.241361403029238, -9.851157263457123, -9.272714169304052, -10.0, -9.851595577937728, -10.0, -9.420229457700302, -9.750079210465744]
Structure 16 dies with -9.750079210465744 fitness
Structure 15 dies with -9.420229457700302 fitness
Structure 14 dies with -10.0 fitness
Structure 13 dies with -9.851595577937728 fitness
Structure 12 dies with -10.0 fitness
Structure 10 dies with -9.851157263457123 fitness
Structure 1 dies with -10.0 fitness
Generation 302 completed. Best modulus: 0.7570422

[-9.387792793480125, -9.27796649425776, -8.408397860946407, -8.407224176760993, -7.059396285044996, -9.241453689350266, -7.631157892400568, -9.27802518783625, -9.24162351738808, -9.272718573335426, -10.0, -10.0, -9.385459539052603, -10.0, -9.33697291420224, -9.539045082515242, -9.698897233282787]
Structure 16 dies with -9.6988972332827

[-9.32516908176103, -9.650718433746663, -9.32633742501558, -9.343137847704572, -9.240317159861213, -9.642569904079124, -9.64306688983358, -9.241225361824036, -9.442882879134078, -9.443344064369072, -9.235025502228718, -9.24923329620633, -9.240262288740206, -9.235031376108605, -9.254320884905733, -10.0, -9.784616210369895]
Structure 16 dies with -9.784616210369895 fitness
Structure 15 dies with -10.0 fitness
Structure 13 dies with -9.235031376108605 fitness
Structure 6 dies with -9.64306688983358 fitness
Structure 2 dies with -9.32633742501558 fitness
Structure 1 dies with -9.650718433746663 fitness
Structure 0 dies with -9.32516908176103 fitness
Generation 315 completed. Best modulus: 0.7607257

[-9.342919886344843, -9.628186740957494, -9.642565275493421, -9.241225361824036, -9.443394664527588, -9.442917399261324, -9.235017230846836, -9.249050301063797, -9.627809185772751, -9.254339745554875, -9.470909913082194, -9.80058166601876, -9.627817770910351, -9.470788953250695, -9.254324172850

[-9.241225361824036, -9.255201809749801, -9.238641070719565, -9.238599547649612, -9.238641070719565, -9.242399884312244, -9.241225361824036, -9.238496094943915, -9.244666788079723, -9.240483646635203, -9.240841999317086, -9.238531016947858, -9.238567550736597, -9.240841999317086, -9.240622828668071, -10.0, -10.0]
Structure 16 dies with -10.0 fitness
Structure 15 dies with -10.0 fitness
Structure 13 dies with -9.240841999317086 fitness
Structure 8 dies with -9.244666788079723 fitness
Structure 4 dies with -9.238641070719565 fitness
Structure 1 dies with -9.255201809749801 fitness
Structure 0 dies with -9.241225361824036 fitness
Generation 327 completed. Best modulus: 0.7607548

[-9.238620371483874, -9.240483646635203, -9.242399884312244, -9.241225361824036, -9.240483646635203, -9.240483646635203, -9.240841999317086, -9.23858014653631, -9.240483646635203, -9.240622828668071, -9.240503810614964, -9.233494409609682, -9.240483646635203, -9.240496298137096, -9.2410400688721, -9.7428924487480

[-9.240042295637103, -9.240556491928087, -9.241225361824036, -9.241225361824036, -9.241225361824036, -9.241225361824036, -9.241225361824036, -9.240761959699828, -9.272675116045379, -9.317100152455806, -10.0, -9.316931512521142, -9.241225361824036, -10.0, -9.278866524386903, -9.502206381293954, -10.0]
Structure 16 dies with -10.0 fitness
Structure 15 dies with -9.502206381293954 fitness
Structure 13 dies with -10.0 fitness
Structure 11 dies with -9.316931512521142 fitness
Structure 10 dies with -10.0 fitness
Structure 9 dies with -9.317100152455806 fitness
Structure 2 dies with -9.241225361824036 fitness
Generation 339 completed. Best modulus: 0.7593547

[-9.240043130764768, -9.240393830838736, -9.241225361824036, -9.241225361824036, -9.241225361824036, -9.241225361824036, -9.241225361824036, -9.273059635664447, -9.241225361824036, -9.278819482507517, -9.551856936651879, -9.269795072526593, -9.241225361824036, -9.552043448670968, -9.241225361824036, -9.635001455869213, -10.0]
Structure 

ValueError: Fewer non-zero entries in p than size

In [74]:
#Print fitness traces to CSV
totalfit = np.transpose(np.array([bestmod, bestdens, bestfit, avg_fitness]))
fitness_df = pd.DataFrame(totalfit)
fitness_df.to_csv('./GA/fitness_traces_ratio_cnn2.csv', header=['best_mod', 'worst_mod', 'best_fit', 'avg_fitness'], index=False)


In [75]:
top_mod_df = pd.DataFrame(top_mod)
top_mod_df.to_csv('./GA/top_mod_ratio_cnn2.csv')

top_fit_df = pd.DataFrame(top_pop)
top_fit_df.to_csv('./GA/top_fit_ratio_cnn2.csv')

all_mod_df = pd.DataFrame(all_mod)
all_mod_df.to_csv('./GA/all_mod_ratio_cnn2.csv')

all_dens_df = pd.DataFrame(all_dens)
all_dens_df.to_csv('./GA/all_dens_ratio_cnn2.csv')